# PyTorch

In [1]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

import os

In [2]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False
log_interval = 100

## Model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Preprocess
### datasets.ImageFolder
label별로 폴더 안에 데이터가 저장되어 있는 경우 사용
- 단, grayscale 데이터는 사용 불가 : https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py#L157

In [4]:
train_path = '../dataset/mnist_png/training'
test_path = '../dataset/mnist_png/testing'

In [5]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [6]:
train_dataset = datasets.ImageFolder(
    root=train_path,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081,))
    ]))

In [7]:
test_dataset = datasets.ImageFolder(
    root=test_path,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081,))
    ]))

In [8]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size = batch_size,
                                          shuffle=True)

In [9]:
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

## Optimization

In [10]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

## Training

In [11]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()
        optimizer.step()  

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()  # batch norm이나 dropout 등을 train mode 변환
    test_loss = 0
    correct = 0
    with torch.no_grad():  # autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()  # pred와 target과 같은지 확인

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

C:\Users\jumin\anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313317
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.242190
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.119130
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.863133
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.300581
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.798503
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.741911
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.553898
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.339050
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.345268

Test set: Average loss: 0.3760, Accuracy: 8915/10000 (89%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.340321
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.203217
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.448332
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.404151
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.385495
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.313359
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.194417
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.240319
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.397526
T